In [1]:
import os
os.chdir('../')

import warnings
warnings.filterwarnings('ignore')

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os

!nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"]="0"

Fri Jun 30 20:05:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    Off  | 00000000:1B:00.0 Off |                    0 |
| 30%   31C    P8    13W / 230W |    779MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A5000    Off  | 00000000:1C:00.0 Off |                  Off |
| 30%   

### Hyperparams

In [5]:
n_mels = 16
n_outputs = 61
n_frames = 400

### Model

In [10]:
from model.model_transformer_vae import Model
device = 'cuda:0'

# Model
model = Model(in_dim=n_mels, h_dim=512, out_dim=n_outputs, z_dim=2)
model = model.to(device)
print('done')

ModuleNotFoundError: No module named 'speechbrain'

In [ ]:
load_dir = '/data/scpark/save/lips/train06.30-1/'
!ls -lt $load_dir

path = load_dir + 'save_' + str(47000)
checkpoint = torch.load(path, map_location=torch.device('cpu'))    
model.load_state_dict(checkpoint['model_state_dict'], strict=True)
print('done')

### Inference

In [ ]:
import librosa

wav_file = '/Storage/speech/tts/kaist-audio-book/wav/남1_동화1/1181.wav'
offset = 0
duration = None
wav, _ = librosa.load(wav_file, sr=24000, offset=offset, duration=duration, res_type='polyphase')
wav = wav / np.max(np.abs(wav))
mel = librosa.feature.melspectrogram(y=wav, sr=24000, n_fft=2048, hop_length=800, win_length=2048, n_mels=n_mels)
mel = np.log10(mel + 1e-5)    
mel_tensor = torch.Tensor(mel).unsqueeze(0).to(device)
print(mel.shape, mel_tensor.shape)

In [ ]:
import librosa.display
import matplotlib.pyplot as plt

model.eval()
with torch.no_grad():
    pred = model.inference(mel_tensor)

plt.figure(figsize=[18, 3])
librosa.display.specshow(pred[0].data.cpu().numpy(), cmap='magma')

plt.figure(figsize=[18, 3])
librosa.display.specshow(mel_tensor[0].data.cpu().numpy(), cmap='magma')